In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  8 15:45:44 2022

@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
#import pyautogui
from selenium.webdriver.common.action_chains import ActionChains
#import shutil
#import glob
import os
import warnings
#import itertools
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#import sys
warnings.filterwarnings("ignore")


In [2]:
#open a new window, load the website and logs in

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')


RL = 'https://events.tc.umn.edu/all'
driver.get(RL)
try:
    driver.maximize_window()
except:
    pass

In [3]:
driver.find_element_by_xpath('/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/p/a[2]').click()


In [4]:
html = driver.page_source
soup = bs(html,'lxml')
allItems= soup.find_all('div',{'class':'lw_cal_event_info'})
umtc=[]


In [6]:
for s in allItems:
    try:
        web=s.find('div',{'class':'lw_events_title heading heading-2'}).find('a')['href']
    except:
        web=s.find('a')['href']
    link='https://events.tc.umn.edu'+web
    #########################
    try:
        tim_tease=s.find('div',{'class':'lw_events_time heading heading-3'}).text
    except:
        tim_tease=''
    ###############################
    try:
        info_tease=s.find('p',{'class':'lw_events_summary'}).text.replace('\n','').strip()
        
    except:
        try:
            info_tease=s.find('span',{'style':'font-weight: 400;'}).text.replace('\n','').strip()
        except:
            info_tease=''
    ##############################################
    if tim_tease=='All Day':
        time_spl=''

    elif tim_tease =='':
        time_spl=''

    elif 'Weekly' in tim_tease:
        time_spl=tim_tease.split('Weekly')[0].strip()

    elif 'Daily' in tim_tease:
        time_spl=tim_tease.split('Daily')[0].strip()

    elif 'Monthly' in tim_tease:
        time_spl=tim_tease.split('Monthly')[0].strip()

    elif 'Yearly' in tim_tease:
        time_spl=tim_tease.split('Yearly')[0].strip()

    elif ('All' and 'to') in tim_tease:
        time_spl=''

    elif 'M-F' in tim_tease:
        time_spl=tim_tease.split('M-F')[0].strip()

    else:
        time_spl=tim_tease
    ##
    if time_spl=='':
        time_st=''
    else:
        zepler=time_spl.replace(' - ',' ').split(' ')
        if len(zepler)<=2:
            time_st={'type': 'general',
                      'start_time':zepler[0],
                      'end_time': '',
                      'timezone': zepler[-1],
                      'days': 'all'}
        elif len(zepler)==3:
            time_st={'type': 'general',
                      'start_time':zepler[0],
                      'end_time': zepler[1],
                      'timezone': zepler[-1],
                      'days': 'all'}

        elif len(zepler)>3:
            time_st={'type': 'general',
                      'start_time':zepler[0],
                      'end_time': zepler[1],
                      'timezone': zepler[-1],
                      'days': 'all'}
        else:
            time_st=''
    
    ###
    if time_st=='':
        time_=''
    else:
        time_=json.dumps(time_st)
    ##############################
    ######################
    ####################
    
    driver.get(link)
    htmln = driver.page_source
    soupn = bs(htmln,'lxml')
    items= soupn.find('div',{'id':'lw_cal_events'})
    lftitems= soupn.find('div',{'id':'lw_cal_event_leftcol'})
    try:
        title=items.find('h2',{'class':'lw_events_title heading heading-2'}).text
    except:
        title=''
    ##############################
    try:
        even=lftitems.find('a',{'class':'lw_join_online umn-button-regular'})['href']
    except:
        even=''
    if even=='':
        event_web=link
    else:
        event_web=even
    ###################################
    #double check
    try:
        if lftitems.find('h3',{'class':'heading heading-3'}).text =='Contact Info:':
            sinzu=lftitems.find('p').text
            if '@' in sinzu:
                sin=sinzu
            else:
                sin=''
        else:
            sin=''
        if sin !='':    
            money=sin.split('\n')
            for mas in money:
                if '@' in mas:
                    mail_=mas
        else:
            mail_=''
    except:
        mail_=''
    ###############################    
    #on_off
    try:
        onner=items.find('div',{'class':'location_marker_container'}).text.replace('\n','')
    except:
        onner=''
    if ('Online'or 'online') in onner:
        on_off='1'
    else:
        on_off='0'
    ###############################
    try:
        event_info=items.find('div',{'class':'location_marker_container'}).text.replace('\n','')
    except:
        if info_tease != '':
            event_info=info_tease
        else:
            event_info=''
            
    ###################################
    zazu=items.find_all('span',{'class':'event_info_label'})
    zeh=len(zazu)
    for n in range(1,zeh+1):
        try:
            if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Time:':
                tim_dur=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
        except NoSuchElementException:
            tim_dur=''

        try:
            if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Date:':
                dat_dur=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
        except NoSuchElementException:
            dat_dur=''

        try:
            if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Location:':
                ven=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
        except NoSuchElementException:
            ven=''
        #price    
        try:
            if driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]/span').text=='Price:':
                price=driver.find_element_by_xpath(f'/html/body/div/div[4]/div/div/div/div[1]/div[2]/div[2]/div/div[2]/div[3]/div[{n}]').text.split(': ')[1]
        except NoSuchElementException:
            price=''
    ####################################
    if on_off=='1':
        venue=''
        city=''
        country=''
    elif on_off=='0':
        venue=ven
        city=ven
        country="USA"
    ####################################
    if dat_dur != '':
        yr=str(date.today().year)
        if '-' in dat_dur:
            sup=dat_dur.split('-')
            spl=[q.strip()+' '+yr for q in sup]
            spl_dt_obj = [datetime.strptime(v, '%b %d %Y') for v in spl]
            date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]
        else:
            sup=[dat_dur, dat_dur]
            spl=[q.strip()+' '+yr for q in sup]
            spl_dt_obj = [datetime.strptime(v, '%b %d %Y') for v in spl]
            date_= [z.strftime('%Y-%m-%d') for z in spl_dt_obj]

    else:
        date_=''

    if date=='':
        start_date=end_date=''
    else:
        start_date=date_[0]
        end_date=date_[1]
    ############################################
    ticket_list=''
    org_name='University of Minnesota Twin Cities'
    org_web='https://twin-cities.umn.edu/'
    org_pro=''
    logo=''    
    sponsor=''
    agendalist=''
    type_=''
    category=''
    Speakerlist=''  
        
        ######################################
    def get_google_map_url(location):
        try:
            google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            google_map_url=driver.current_url
            return(google_map_url)
        except Exception as e:
            print(e)
            return("")
        
    if venue!='':
        googlePlaceUrl=get_google_map_url(venue)
    else:
        googlePlaceUrl=''
        ############
    umtc.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                 org_pro,org_name,org_web,logo,sponsor,agendalist,
                 type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                 Speakerlist,on_off])

In [7]:
tim_tease

'12:00pm CDT'

In [8]:
umtc_dict=umtc


umtc_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=umtc_dict)
umtc_df.to_csv("umtc.tsv", sep = '\t',index=False)
